# Limpeza_dados_desafio

# ■ Limpeza e Validação de Dados — Desafio de Data Cleaning

Este notebook realiza o processo completo de **limpeza e validação de dados** para o dataset
`desafio_limpeza`,
seguindo o dicionário de variáveis proposto na atividade.

Cada etapa é documentada e utiliza boas práticas de análise e pré-processamento de dados.

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime


## 2 Carregamento do Dataset

In [ ]:
# Substituir pelo caminho correto do arquivo CSV, por exemplo:
# df = pd.read_csv('dados_desafio_limpeza.csv')
# Para fins de demonstração, vamos criar um dataset simulado:
dados = {
'id': range(1, 11),
'nome': ['Ana Souza', 'Bruno Lima', 'Carlos Silva', 'daniela martins',
'EVA PEREIRA',
'Felipe', 'Gabriel Souza', 'helena costa', 'Igor Santos', None],
'idade': [25, 42, 17, 102, 35, -1, 55, 28, 90, 40],
'renda_mensal': [3500, 8200, None, 5000, -1500, 4200, 3000, 0, 2700,
5400],
'estado_civil': ['Solteiro', 'Casado', 'Divorciado', 'Solteira', None,
'CASADO', 'viuvo', 'Divorciado', 'Solteiro', 'Casado'],
'uf': ['SP', 'RJ', 'MG', 'XX', 'BA', 'RS', None, 'PE', 'DF', 'sp'],
'data_cadastro': ['2022-01-10', '2021/03/05', '20-04-2020', '2019-13-01',
None, '2020-12-30', '2021-02-29', '2023-06-01', '2024-01-15',
'2020-10-10'],
'score_credito': [800, 450, 1200, -10, 600, 700, 300, None, 500, 1000],
'categoria_produto': ['A', 'B', 'C', 'D', None, 'a', 'B', 'C', 'b', 'A'],
'qtd_compras_12m': [10, 5, -2, 20, 0, 15, None, 8, 7, 3],
'email': ['ana@email.com', 'bruno@gmail', 'carlos.silva@empresa.com',
'dani.martins@site',
None, 'felipe@@mail.com', 'gabriel@outlook.com',
'helena.costa@gmail.com',
'igorsantos', 'eva@p@gmail.com']
}
df = pd.DataFrame(dados)
df.head()


# ## 3 Análise Inicial


In [ ]:
df.info()
df.describe(include='all')

# 4 Funções auxiliares para limpeza e validação


In [ ]:
def validar_email(email):
pattern = r'^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$'
return bool(re.match(pattern, str(email)))
def validar_uf(uf):
ufs = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG','
PA','PB','PR',
'PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO']
return uf in ufs
def corrigir_nome(nome):
if pd.isna(nome): return None
return ' '.join([n.capitalize() for n in nome.split()])
def corrigir_estado_civil(valor):
if pd.isna(valor): return None
valor = valor.strip().lower()
mapa = {
'solteiro': 'Solteiro', 'solteira': 'Solteiro',
'casado': 'Casado', 'casada': 'Casado',
'divorciado': 'Divorciado', 'divorciada': 'Divorciado',
'viuvo': 'Viúvo', 'viúva': 'Viúvo'
}
return mapa.get(valor, None)

# 5 Aplicando Limpeza Passo a Passo

Coluna `nome`

In [ ]:
df['nome'] = df['nome'].apply(corrigir_nome)
df['nome'].fillna('Não informado', inplace=True)
df.head()


Coluna `idade`

In [ ]:
df.loc[(df['idade'] < 15) | (df['idade'] > 100), 'idade'] = np.nan
df['idade'].fillna(df['idade'].median(), inplace=True)


Coluna `renda_mensal`

In [ ]:
df.loc[df['renda_mensal'] <= 0, 'renda_mensal'] = np.nan
df['renda_mensal'].fillna(df['renda_mensal'].median(), inplace=True)


Coluna `estado_civil`

In [ ]:
df['estado_civil'] = df['estado_civil'].apply(corrigir_estado_civil)
df['estado_civil'].fillna('Solteiro', inplace=True)


Coluna `uf`


In [ ]:
df['uf'] = df['uf'].str.upper()
df.loc[~df['uf'].apply(validar_uf), 'uf'] = np.nan
df['uf'].fillna('SP', inplace=True)


 Coluna `data_cadastro`


In [ ]:
def validar_data(data):
try:
return pd.to_datetime(data, errors='coerce')
except:
return pd.NaT
df['data_cadastro'] = df['data_cadastro'].apply(validar_data)
df['data_cadastro'].fillna(pd.Timestamp('2022-01-01'), inplace=True)

 Coluna `score_credito`


In [ ]:
df.loc[(df['score_credito'] < 0) | (df['score_credito'] > 1000),
'score_credito'] = np.nan
df['score_credito'].fillna(df['score_credito'].mean(), inplace=True)


Coluna `categoria_produto`

In [ ]:
df['categoria_produto'] = df['categoria_produto'].str.upper()
df.loc[~df['categoria_produto'].isin(['A', 'B', 'C']),
'categoria_produto'] = np.nan
df['categoria_produto'].fillna('C', inplace=True)


Coluna `qtd_compras_12m`


In [ ]:
df.loc[df['qtd_compras_12m'] < 0, 'qtd_compras_12m'] = 0
df['qtd_compras_12m'].fillna(df['qtd_compras_12m'].median(),
inplace=True)


Coluna `email`

In [ ]:
df.loc[~df['email'].apply(validar_email), 'email'] = np.nan
df['email'].fillna('email_invalido@dominio.com', inplace=True)

# 6 Verificação Final


In [ ]:
df.info()
df.head(10)


# 7 Exportação dos Dados Limpados

In [ ]:
df.to_csv('dados_desafio_limpeza_tratado.csv', index=False)
print('Arquivo salvo com sucesso!')


# Conclusão


# Todos os campos foram validados conforme o dicionário de variáveis.
# Valores inconsistentes foram tratados ou substituídos.
# E-mails e datas passaram por verificação de formato.
# O dataset final está pronto para uso em análises ou modelagem de crédito.